In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
#from keras.applications.resnet import ResNet50
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/MyDrive/deep_fake_image_detection/Image_segregated/train'
valid_path = '/content/drive/MyDrive/deep_fake_image_detection/Image_segregated/val'

# add preprocessing layer to the front of VGG(include=False means donot add the last layer)
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)



58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
import os
import cv2
labels = ['fake', 'real']
img_size = 224
def get_data(data_dir):
    data = []
    for label in labels:
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img))[...,::-1] #convert BGR to RGB format
                resized_arr = cv2.resize(img_arr, (img_size, img_size)) # Reshaping images to preferred size
                data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
    return np.array(data)

In [ ]:
train = get_data(r'/content/drive/MyDrive/deep_fake_image_detection/Image_segregated/train')
test = get_data(r'/content/drive/MyDrive/deep_fake_image_detection/Image_segregated/test')

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (1999, 2) + inhomogeneous part.

In [ ]:
import os
import cv2
import numpy as np

def load_images_and_labels(folder_path):
    images = []
    labels = []

    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)

            # Resize or preprocess the image as needed
            # img = cv2.resize(img, (width, height))

            label = int(filename.startswith("fake"))  # Assuming "fake" corresponds to label 0, adjust accordingly
            images.append(img)
            labels.append(label)

    return np.array(images), np.array(labels)

# Example usage
train_folder = "/content/drive/MyDrive/deep_fake_image_detection/Image_segregated_2/train"
X_train, y_train = load_images_and_labels(train_folder)

# Now you have X_train (images) and y_train (labels) ready to use


In [ ]:
X_train.shape

(1999, 224, 224, 3)

In [ ]:
y_train.shape

(1999,)

In [ ]:
# Example usage
val_folder = "/content/drive/MyDrive/deep_fake_image_detection/Image_segregated_2/val"
X_val, y_val = load_images_and_labels(val_folder)


In [ ]:
X_val.shape

(810, 224, 224, 3)

In [ ]:
y_val.shape

(810,)

In [ ]:
test_folder = "/content/drive/MyDrive/deep_fake_image_detection/Image_segregated_2/test"
X_test, y_test = load_images_and_labels(test_folder)


In [ ]:
X_test.shape

(819, 224, 224, 3)

In [ ]:
y_test.shape

(819,)

In [ ]:
# Normalize the data
X_train = np.array(X_train).astype('float32') / 255
X_val = np.array(X_val).astype('float32') / 255

In [ ]:
# don't train existing weights
for layer in vgg.layers:
    layer.trainable = False

In [ ]:
folders = glob('/content/drive/MyDrive/deep_fake_image_detection/Image_segregated/train')
len(folders)

1

In [ ]:
folders = glob('/content/drive/MyDrive/deep_fake_image_detection/Image_segregated/train')


# our layers - you can add more if you want
x = Flatten()(vgg.output)
# x = Dense(1000, activation='relu')(x)
prediction = Dense(1, activation='sigmoid')(x)

# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

# view the structure of the model
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
# tell the model what cost and optimization method to use
model.compile(
  loss='binary_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:

# useful for getting number of classes
folders = glob('/content/drive/MyDrive/deep_fake_image_detection/Image_segregated/train/*')





from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/deep_fake_image_detection/Image_segregated/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/deep_fake_image_detection/Image_segregated/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')



Found 1999 images belonging to 2 classes.
Found 809 images belonging to 2 classes.


In [ ]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

<ipython-input-20-dcaa1ca38143>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


KeyboardInterrupt: 